In [9]:
import os, sys, inspect

import numpy as np
import pandas as pd


import matplotlib
from matplotlib import pyplot as plt

import collections
import time
import datetime

import re

import requests
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException

working_dir = os.path.dirname(
    os.path.abspath(
        inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(working_dir)
sys.path.insert(0, parent_dir)

from scripts_python import scraper as scr

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
driver = scr.get_web_driver()

1. Chrome - Help - About Google Chrome
2. check version; e.g. `Version 79.0.3945.117 (Official Build) (64-bit)`
3. [download relevant ChromeDriver](https://chromedriver.chromium.org/downloads)

In [11]:
def load_glassdoor_page(
        driver: selenium.webdriver.chrome.webdriver.WebDriver,
        url: str = 'glassdoor.com',
        ):

    print(url)
    driver.get(url)

In [12]:
load_glassdoor_page(driver)

glassdoor.com


InvalidArgumentException: Message: invalid argument
  (Session info: chrome=79.0.3945.117)


In [13]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('log-level=3')
browser = webdriver.Chrome(options=chrome_options)

WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home
